## Project 5: Vehicle Detection and Tracking

### Load image paths and explore dataset

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from skimage.feature import hog
from skimage import color, exposure
%matplotlib inline

# images are divided up into vehicles and non-vehicles
car_images_udacity = glob.glob(r'C:\Users\mohar\Desktop\Udacity\vehicles\*\*.png')
notcar_images_udacity = glob.glob(r'C:\Users\mohar\Desktop\Udacity\non-vehicles\*\*.png')

cars = []
notcars = []

for image in car_images_udacity:
    cars.append(image)

for image in notcar_images_udacity:
    notcars.append(image)
        
# Define a function to return some characteristics of the dataset 
def data_look(car_list, notcar_list):
    data_dict = {}
    # Define a key in data_dict "n_cars" and store the number of car images
    data_dict["n_cars"] = len(car_list)
    # Define a key "n_notcars" and store the number of notcar images
    data_dict["n_notcars"] = len(notcar_list)
    # Read in a test image, either car or notcar
    test_img = mpimg.imread(car_list[0])
    # Define a key "image_shape" and store the test image shape 3-tuple
    data_dict["image_shape"] = test_img.shape
    # Define a key "data_type" and store the data type of the test image.
    data_dict["data_type"] = test_img.dtype
    # Return data_dict
    return data_dict
    
data_info = data_look(cars, notcars)

print('Your function returned a count of', 
      data_info["n_cars"], ' cars and', 
      data_info["n_notcars"], ' non-cars')
print('of size: ',data_info["image_shape"], ' and data type:', 
      data_info["data_type"])

Your function returned a count of 4050  cars and 8968  non-cars
of size:  (64, 64, 3)  and data type: float32


### Since the number of notcar images is more than car images we augment the car class using crowdai's dataset. The images from crowdai's dataset have been cropped as per bounding boxes and resized to (64, 64, 3). The processed images are stored in a folder called resized. The code can be found in Augment_dataset.ipynb

In [2]:
car_images_crowdai = glob.glob(r'C:\Users\mohar\Desktop\Udacity\Self-Driving-Car-NanoDegree-Udacity\\
Term1-Computer-Vision-and-Deep-Learning\Project5-Vehicle-Detection-and-Tracking\resized\*.jpg')
# load image path randomly from resized images to augment car class
for i in range(len(notcars) - len(cars)):
        np.random.seed(0) # generate same random numbers everytime to avoid issues due to variation in training data later
        cars.append(np.random.randint(0, len(car_images_crowdai)))

### Verify that each class has equal number of training examples

In [3]:
data_info = data_look(cars, notcars)

print('Your function returned a count of', 
      data_info["n_cars"], ' cars and', 
      data_info["n_notcars"], ' non-cars')
print('of size: ',data_info["image_shape"], ' and data type:', 
      data_info["data_type"])


Your function returned a count of 8968  cars and 8968  non-cars
of size:  (64, 64, 3)  and data type: float32


### Helper functions

In [4]:
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [5]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [6]:
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))

In [8]:
def extract_features(imgs, cspace='RGB', spatial_size=(16, 16), hist_bins=32, hist_range=(0, 256),
                     norient=9, hog_pix_cell=(8, 8), hog_cell_block=(2,2)):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        else: feature_image = np.copy(image)      
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() to get histogram color features
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        # Apply get_hog_features() to get hog features
        hog_features = get_hog_features(feature_image, orient=norient, pix_per_cell=hog_pix_cell,
                                         cell_per_block=hog_cell_block)
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features, hog_features)))
    # Return list of feature vectors
    return features

### Building a classifier using linear support vector machine (svm)